In [201]:
import cv2
import tensorflow as tf
import numpy as np
from PIL import Image
def read_image(path):
    return np.array(Image.open(path))
def show(image):
    image = image.astype('uint8')
    im = Image.fromarray(image)
    im.show()

In [193]:
def split(image, ksizes, strides):
    images = tf.extract_image_patches(image, ksizes=ksizes, strides=strides, padding='SAME', rates=[1,1,1,1])
    new_shape = [-1, *images.get_shape().as_list()[1:3], *ksizes[1:3], image.get_shape().as_list()[-1]]
    images = tf.reshape(images, new_shape)
    return images

def join(image, images, ksizes, strides):
    def f(images):
        cx = ksizes[1] // 2
        cy = ksizes[2] // 2
        x1 = cx - strides[1]//2
        x2 = x1+strides[1]
        y1 = cy - strides[2]//2
        y2 = y1+strides[2]        
        images_croped = images[:,:,:,x1:x2, y1:y2,:]
        return images_croped
    split_image = tf.split(images, image.get_shape().as_list()[-1], axis=-1)
    outputs = [f(img) for img in split_image]
    outputs = tf.concat(outputs, axis=-1)
    return tf.transpose(outputs, perm=[0,1,3,2,4,5])

In [198]:
import math
ksizes=[1,100, 200,1]
strides = [1, 100, 30, 1]

np_image = read_image('0.png')

origin_shape = np_image.shape[:2]
new_h = math.ceil(origin_shape[0]/ksizes[1])*ksizes[1]
new_w = math.ceil(origin_shape[1]/ksizes[2])*ksizes[2]
np_image = cv2.resize(np_image, (new_w, new_h))
np_image = np.expand_dims(np_image, 0)
np_image = np_image[:,:,:,:1]

In [199]:
image = tf.placeholder(tf.float32, [None, *np_image.shape[1:]])
images = split(image, ksizes, strides)
image_out = join(image, images, ksizes, strides)
sess = tf.Session()
outputs = sess.run(image_out, {image:np_image})
output_reshape = outputs.reshape(np_image.shape[1:4])
output_reshape = cv2.resize(output_reshape, (origin_shape[1], origin_shape[0]))

In [200]:
show(output_reshape)

In [ ]:
path = 